## Figure 7

In [ ]:
%pylab notebook

from scipy.optimize import curve_fit

import sys, os, importlib, warnings, pandas as pd, seaborn as sns
warnings.filterwarnings("ignore")

sys.path.append('/Users/alexisdevitre/Documents/code/hts-irradiation')
import hts_fitting as hts, hts_fitfunctions as ff, hts_misc as misc, default_figure_style as dfs
for lib in [hts, dfs, ff, misc]:
    importlib.reload(lib)


label_suppression = 'Suppression $\mathrm{1 - I_c^{ON} / I_c^{OFF}}$'
label_degradation = 'Degradation $I_c^{OFF} / I_c^{0}$'

#################################
########## Correction ###########
#################################

def sqrt4(x, a, b, c, d):
    return a*np.sqrt(c*x+d)+b

#################################
########## Load data ############
#################################

masterfile = '../data/data-master-linear.xlsx'
cols = [0, 1, 12, 13, 14, 19, 23, 25, 27, 29, 39, 41, 43, 45, 52, 53, 54, 55, 56, 57, 58, 59]
names = [
    'tapeid', 
    'anneal',
    'energy', 
    'power', 
    'powerError',
    'ic0',
    'icoff',
    'noff',
    'icon',
    'non',
    
    'tHTS_OFF',
    'tTAR_OFF',
    'tHTS_ON',
    'tTAR_ON',
    
    'tREBCO',
    'degradation',
    'degradationError',
    'suppression', 
    'suppressionError',
    'suppression_corrected',
    'suppressionError_corrected',
    'outlier' 
]
df = pd.read_excel(masterfile, sheet_name='data', usecols=cols, names=names, skiprows=1)

#################################
########## Filter data ##########
#################################
cut = df.tTAR_OFF <= 20
cut &= (df.suppression != np.nan) & (df.suppression > 0) & (df.suppressionError > 0)
cut &= df.outlier != 1
data = df[cut]

print('\n\nTotal datapoints', data.energy.count())
for tapeid in data.tapeid.unique():
    print('\nDatapoints for', tapeid, data[data.tapeid==tapeid].energy.count())

df.columns
outputDirectory = '../figures/raw/'


fitfunction = ff.linear

percent = 100 # switch suppression values to percentages

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
labels = {
    '800': '12x damage rate (800 keV)',
    '1200': '4x  damage rate (1200 keV)',
    '2400': '1x  damage rate (2400 keV)',
    '150': 'No damage (150 keV)'
}


d = data[(data.tapeid == 'f29')]
d['dpaRate'] = d['energy'].replace([150, 800, 1200, 2400], [0, 12.5, 4.4, 1])

power_cut = (34 < d.power) & (d.power < 36)
x, ex = d.dpaRate[power_cut], d.powerError[power_cut]
y, ey = d.suppression_corrected[power_cut], d.suppressionError[power_cut]

for e, p, l in zip([150, 2400, 1200, 800], [0, 1, 2, 3], ['0', '1', '4.4', '12.5']):
    cut, c = (d.energy == e), dfs.colors_energies[str(e)]
    ydata = y[cut]*percent
    boxprops = dict(facecolor=c)
    ax.boxplot(ydata, positions=[p], labels=[l], patch_artist=True, showfliers=False, boxprops=boxprops)
    
    xjitter = np.random.normal(p-.2, 0.02, size=len(y[cut]))
    plt.scatter(xjitter, ydata, alpha=0.6, color=c, s=20)


d = data[(data.tapeid == 'f33')]
d['dpaRate'] = d['energy'].replace([150, 800, 1200, 2400], [0, 12.5, 4.4, 1])

power_cut = (34 < d.power) & (d.power < 36)
x, ex = d.dpaRate[power_cut], d.powerError[power_cut]
y, ey = d.suppression_corrected[power_cut], d.suppressionError[power_cut]

for e, p, l in zip([150, 2400, 1200, 800], [0, 1, 2, 3], ['0', '1', '4.4', '12.5']):
    cut, c = (d.energy == e), dfs.colors_energies[str(e)]
    ydata = y[cut]*percent
    boxprops = dict(facecolor=c)
    ax.boxplot(ydata, positions=[p], labels=[l], patch_artist=True, showfliers=False, boxprops=boxprops)
    
    xjitter = np.random.normal(p-.2, 0.02, size=len(y[cut]))
    plt.scatter(xjitter, ydata, alpha=0.6, color=c, s=20)
    
ax.set_ylim(0, 35)
ax.set_ylabel('Corrected Suppression [%]') # $\mathrm{1 - I_c^{ON}\, /\, I_c^{OFF}}$
ax.set_xlabel('Atomic displacement rate in the\nREBCO layer [$\mathrm{10^{-3}}$ mdpa/s]')

fig.tight_layout()




fitfunction = ff.affine

percent = 100 # switch suppression values to percentages

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
labels = {
    '800': '12x damage rate (800 keV)',
    '1200': '4x  damage rate (1200 keV)',
    '2400': '1x  damage rate (2400 keV)',
    '150': 'No damage (150 keV)'
}


for tape in ['f29', 'f33']:
    d = data[(data.tapeid == tape)]# & (data.anneal == 1)]
    x, ex = d.power, d.powerError
    y, ey = d.suppression_corrected, d.suppressionError
    #z, ey = d.suppression, d.suppressionError

    popt, pcov = curve_fit(fitfunction, x, y)
    xsmooth = np.linspace(0, 150, 10000)
    ysmooth = fitfunction(xsmooth, *popt)

    ax.plot(xsmooth, percent*ysmooth, linewidth=4, alpha=.3, color='gray')

    for e, m in zip([800, 1200, 2400, 150], ['^', 's', 'v', 'o']):
        cut = (d.energy == e)
        c = mec = mfc = dfs.colors_energies[str(e)]
        if tape == 'f29':
            l = labels[str(e)]
        else:
            l = ''
        if e == 150:
            mfc = 'white'
        elif e == 800:
            zorder = 20
        else:
            zorder = 1

        ax.errorbar(x[cut], percent*y[cut], xerr=ex[cut], yerr=ey[cut], linestyle='None', marker='None', color=c, alpha=.5, zorder=1)
        ax.plot(x[cut], percent*y[cut], marker=m, linestyle='None', mec=mec, mfc=mfc, markersize=8, label='{:>}'.format(l), zorder=zorder)

ax.set_ylabel('Corrected Suppression [%]') # $\mathrm{1 - I_c^{ON}\, /\, I_c^{OFF}}$
ax.set_xlabel('Ion Beam Power [mW]')
ax.legend(loc='upper left')
ax.set_ylim(0, percent*.35)
ax.set_xlim(0, 50)
fig.tight_layout()
ax.set_rasterized(True)
plt.savefig(outputDirectory+'150v2400v1200v800keV.svg', facecolor='white')


# Figure 9

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
fitfunction = ff.affine
xsmooth = np.linspace(0, 300, 1000)
xx, yy, ii, ee, aa, tt, yn = np.array([]), np.array([]), np.array([]), np.array([]), np.array([]), np.array([]), np.array([])


rho0 = 0.2525580531927326 # interface thermal resistance for ffj28


# There are two sets for f29 and f37 because the first anneal 
# changes the thermal coupling (response?) of the tape
alldata = []
allTapes = np.array(['f20', 'f23', 'f28', 'f29', 'f33', 'f34', 'f37'])
palette = sns.color_palette('viridis', 9)

for tape in allTapes:
    d = data[(data.tapeid == tape) & (data.tREBCO-data.tTAR_ON > 0) & (data.anneal == 1)]
    alldata.append(d)
    
    if (tape == 'f29')|(tape == 'f37'):
        d = data[(data.tapeid == tape) & (data.tREBCO-data.tTAR_ON > 0) & (data.anneal > 1)]
        alldata.append(d)

for i, d in enumerate(alldata):
    tape = d.tapeid.iloc[0]
    x, ex = d.power, d.powerError
    y, ey = d.tREBCO-d.tTAR_ON, np.zeros_like(d.powerError)

    popt, pcov = curve_fit(ff.affine, x, y)
    handles, labels = plt.gca().get_legend_handles_labels()
    label = tape.replace('f', 'ffj')
    if label in labels:
        label += '$\mathrm{^{a}}$'
        
    if tape == 'f28':
        print('ffj28 thermal resistance', popt[0])
        ax.errorbar(x, y, xerr=ex, linestyle='None', marker='+', markersize=8, color=palette[i], label=label)
    else:
        ax.errorbar(x, rho0*y/popt[0], xerr=ex, linestyle='None', marker='+', markersize=8, color=palette[i], label=label)
    xx = np.append(xx, x)
    yy = np.append(yy, rho0*y/popt[0])
    ee = np.append(ee, d.energy)
    ii = np.append(ii, d.index)
    tt = np.append(tt, d.tapeid)
    aa = np.append(aa, d.anneal)
    yni = []
    for ei in d.energy:
        if ei == 150:
            yni.append(1)
        else:
            yni.append(0)
    yn = np.append(yn, yni)
  

#parentDirectory = '/Users/alexisdevitre/Desktop/1200keV/'
#modelmin = pd.read_csv(parentDirectory+'f28_1200keV_deltaT_Apiezon=10um+Sensorpos=4.96mm.txt', names=['ibeam', 'tREBCO'], usecols=[0, 3], delim_whitespace=True, skiprows=5)
#modelmax = pd.read_csv(parentDirectory+'f28_1200keV_deltaT_Apiezon=16um+Sensorpos=4.96mm.txt', names=['ibeam', 'tREBCO'], usecols=[0, 3], delim_whitespace=True, skiprows=5)

#popt_min, _ = curve_fit(ff.affine, modelmin.ibeam*1200/1e3, modelmin.tREBCO)
#popt_max, _ = curve_fit(ff.affine, modelmax.ibeam*1200/1e3, modelmax.tREBCO)
    

#ax.fill_between(xsmooth, ff.affine(xsmooth, *popt_min), ff.affine(xsmooth, *popt_max), color='lightgray', alpha=.2)
#ax.plot(xsmooth, ff.affine(xsmooth, *popt_min), linewidth=1, linestyle=':', color='gray', alpha=.3)
#ax.plot(xsmooth, ff.affine(xsmooth, *popt_max), linewidth=1, linestyle=':', color='gray', alpha=.3)



ax.legend(loc='upper left', ncols=2, columnspacing=.5)
ax.set_xlabel('Ion Beam Power [mW]')
ax.set_ylabel(r'$\mathrm{\frac{\rho_{ffj28}}{\rho} \cdot (T_{REBCO} - T_{Cold mass})}$ [K]')
ax.set_xlim(0, 300)
ax.set_ylim(0, 80)
fig.tight_layout()
plt.savefig(outputDirectory+'thermal-resistance-normalized.svg', format='svg', facecolor='white')